<a href="https://colab.research.google.com/github/tewei0328/teach-programming/blob/main/Python_%E6%99%82%E9%96%93%E5%BA%8F%E5%88%97%E5%AF%A6%E5%81%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python 時間序列實做

https://www.finlab.tw/python-%E6%99%82%E9%96%93%E5%BA%8F%E5%88%97%E5%AF%A6%E4%BD%9C%EF%BC%81/

In [26]:
import requests
from io import StringIO
import pandas as pd
import numpy as np

def crawl_price(date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '})
                                        for i in r.text.split('\n')
                                        if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = ret.set_index('證券代號')
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    return ret

In [27]:
import datetime
import time

data = {}
n_days = 2
date = datetime.datetime.now()
fail_count = 0
allow_continuous_fail_count = 5
while len(data) < n_days:

    print('parsing', date)
    # 使用 crawPrice 爬資料
    try:
        # 抓資料
        data[date.date()] = crawl_price(date)
        print('success!')
        fail_count = 0
    except:
        # 假日爬不到
        print('fail! check the date is holiday')
        fail_count += 1
        if fail_count == allow_continuous_fail_count:
            raise
            break

    # 減一天
    date -= datetime.timedelta(days=1)
    time.sleep(10)

parsing 2023-07-18 03:32:32.391829
fail! check the date is holiday
parsing 2023-07-17 03:32:32.391829
success!
parsing 2023-07-16 03:32:32.391829
fail! check the date is holiday
parsing 2023-07-15 03:32:32.391829
fail! check the date is holiday
parsing 2023-07-14 03:32:32.391829
success!


In [28]:
type(data)

dict

In [29]:
data[datetime.date(2023,7,17)]

,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
證券代號,,,,,,,,,,,,,,,,
1101,台泥,19635471,"11,288",722831068,37.00,37.10,36.70,36.75,-,0.25,36.70,"2,088",36.75,16,48.36,NaN
1101B,台泥乙特,5068,8,243046,48.00,48.10,47.85,48.10,+,0.10,47.90,2,48.10,4,0.00,NaN
1102,亞泥,6387281,"2,930",280789057,43.95,44.15,43.90,43.90,NaN,0.00,43.90,275,43.95,23,12.80,NaN
1103,嘉泥,716343,383,15268027,21.25,21.50,21.15,21.40,+,0.15,21.40,12,21.45,24,0.00,NaN
1104,環泥,2705864,"1,048",84074842,31.00,31.40,30.80,31.15,+,0.60,31.10,68,31.15,53,11.45,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,新麗,376553,205,8063705,21.20,21.70,21.15,21.50,+,0.30,21.50,6,21.55,4,4.77,NaN
9945,潤泰新,3929345,"3,067",138994400,35.55,35.60,35.15,35.45,-,0.10,35.40,126,35.45,98,20.03,NaN
9946,三發地產,73263,85,1006054,13.80,13.85,13.65,13.70,-,0.15,13.65,18,13.75,9,35.13,NaN


In [30]:
data

{datetime.date(2023, 7, 17):        證券名稱      成交股數    成交筆數        成交金額     開盤價     最高價     最低價     收盤價  \
 證券代號                                                                        
 1101     台泥  19635471  11,288   722831068   37.00   37.10   36.70   36.75   
 1101B  台泥乙特      5068       8      243046   48.00   48.10   47.85   48.10   
 1102     亞泥   6387281   2,930   280789057   43.95   44.15   43.90   43.90   
 1103     嘉泥    716343     383    15268027   21.25   21.50   21.15   21.40   
 1104     環泥   2705864   1,048    84074842   31.00   31.40   30.80   31.15   
 ...     ...       ...     ...         ...     ...     ...     ...     ...   
 9944     新麗    376553     205     8063705   21.20   21.70   21.15   21.50   
 9945    潤泰新   3929345   3,067   138994400   35.55   35.60   35.15   35.45   
 9946   三發地產     73263      85     1006054   13.80   13.85   13.65   13.70   
 9955     佳龍    261864     185     6103462   23.30   23.55   23.20   23.30   
 9958    世紀鋼  15709344   7,503  2339

In [31]:
close = pd.DataFrame({k:d['收盤價'] for k,d in data.items()}).transpose()
close.index = pd.to_datetime(close.index)
close

證券代號,1101,1101B,1102,1103,1104,1108,1109,1110,1201,1203,...,9940,9941,9941A,9942,9943,9944,9945,9946,9955,9958
2023-07-17,36.75,48.10,43.90,21.40,31.15,16.50,17.30,19.55,19.80,41.45,...,29.15,218.50,50.20,149.00,91.30,21.50,35.45,13.70,23.30,149.00
2023-07-14,37.00,48.00,43.90,21.25,30.55,16.55,17.30,19.35,19.80,41.20,...,29.50,215.00,50.20,151.50,92.10,21.20,35.55,13.85,23.50,144.00


In [32]:
data.items()

dict_items([(datetime.date(2023, 7, 17),        證券名稱      成交股數    成交筆數        成交金額     開盤價     最高價     最低價     收盤價  \
證券代號                                                                        
1101     台泥  19635471  11,288   722831068   37.00   37.10   36.70   36.75   
1101B  台泥乙特      5068       8      243046   48.00   48.10   47.85   48.10   
1102     亞泥   6387281   2,930   280789057   43.95   44.15   43.90   43.90   
1103     嘉泥    716343     383    15268027   21.25   21.50   21.15   21.40   
1104     環泥   2705864   1,048    84074842   31.00   31.40   30.80   31.15   
...     ...       ...     ...         ...     ...     ...     ...     ...   
9944     新麗    376553     205     8063705   21.20   21.70   21.15   21.50   
9945    潤泰新   3929345   3,067   138994400   35.55   35.60   35.15   35.45   
9946   三發地產     73263      85     1006054   13.80   13.85   13.65   13.70   
9955     佳龍    261864     185     6103462   23.30   23.55   23.20   23.30   
9958    世紀鋼  15709344   7,503  2339

In [33]:
{k:d['收盤價'] for k,d in data.items()}

{datetime.date(2023, 7, 17): 證券代號
 1101      36.75
 1101B     48.10
 1102      43.90
 1103      21.40
 1104      31.15
           ...  
 9944      21.50
 9945      35.45
 9946      13.70
 9955      23.30
 9958     149.00
 Name: 收盤價, Length: 1016, dtype: object,
 datetime.date(2023, 7, 14): 證券代號
 1101      37.00
 1101B     48.00
 1102      43.90
 1103      21.25
 1104      30.55
           ...  
 9944      21.20
 9945      35.55
 9946      13.85
 9955      23.50
 9958     144.00
 Name: 收盤價, Length: 1016, dtype: object}

In [34]:
close = pd.DataFrame({k:d['收盤價'] for k,d in data.items()}).transpose()

In [35]:
close


證券代號,1101,1101B,1102,1103,1104,1108,1109,1110,1201,1203,...,9940,9941,9941A,9942,9943,9944,9945,9946,9955,9958
2023-07-17,36.75,48.10,43.90,21.40,31.15,16.50,17.30,19.55,19.80,41.45,...,29.15,218.50,50.20,149.00,91.30,21.50,35.45,13.70,23.30,149.00
2023-07-14,37.00,48.00,43.90,21.25,30.55,16.55,17.30,19.35,19.80,41.20,...,29.50,215.00,50.20,151.50,92.10,21.20,35.55,13.85,23.50,144.00


In [36]:
close.index = pd.to_datetime(close.index)

In [37]:
close


證券代號,1101,1101B,1102,1103,1104,1108,1109,1110,1201,1203,...,9940,9941,9941A,9942,9943,9944,9945,9946,9955,9958
2023-07-17,36.75,48.10,43.90,21.40,31.15,16.50,17.30,19.55,19.80,41.45,...,29.15,218.50,50.20,149.00,91.30,21.50,35.45,13.70,23.30,149.00
2023-07-14,37.00,48.00,43.90,21.25,30.55,16.55,17.30,19.35,19.80,41.20,...,29.50,215.00,50.20,151.50,92.10,21.20,35.55,13.85,23.50,144.00


In [38]:
type(close)

pandas.core.frame.DataFrame

In [39]:
close['1101']

2023-07-17    36.75
2023-07-14    37.00
Name: 1101, dtype: object

In [40]:
type(close['1101'])

pandas.core.series.Series

In [41]:
close['1101'][0]

'36.75'

In [42]:
close.head()

證券代號,1101,1101B,1102,1103,1104,1108,1109,1110,1201,1203,...,9940,9941,9941A,9942,9943,9944,9945,9946,9955,9958
2023-07-17,36.75,48.10,43.90,21.40,31.15,16.50,17.30,19.55,19.80,41.45,...,29.15,218.50,50.20,149.00,91.30,21.50,35.45,13.70,23.30,149.00
2023-07-14,37.00,48.00,43.90,21.25,30.55,16.55,17.30,19.35,19.80,41.20,...,29.50,215.00,50.20,151.50,92.10,21.20,35.55,13.85,23.50,144.00
